In [ ]:
#Importando bibliotecas 
import pandas as pd
import numpy as np
import mysql.connector
import os
from dotenv import load_dotenv

#Carreagndo váriaveis do arquivo .env
load_dotenv()

#Obtendo váriaveis com informações de conexão do BD
db_host = os.getenv('DB_HOST')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')

#Criando conexão com Banco de Dados
connection = mysql.connector.connect(
    host=db_host,
    user=db_user,
    password=db_password,
    database=db_name
)

#Iniciando o cursor para realizar consultas SQL
cursor = connection.cursor()

#Definindo o caminho, onde está o arquivo .csv
caminho = os.getenv('caminho_importacao')

#Iterando entre os arquivos.csv que contém no caminho citado acima
part_files = [caminho + "part -{}.csv".format(i) for i in range(25, 31)]

#Iniciando a iteração em cada arquivo contido no caminho citado anteriormente, reiniciando até o ultimo arquivo
for part_file in part_files:
    file_path = os.path.join(".", part_file) #Combinando caminho do arquivo com o diretório atual.
    print("Lendo arquivo:", part_file) #Exibindo mensagem de Leitura
    df_part = pd.read_csv(file_path, sep=';') #Realizando a leitura do DF
    
    #Convertendo coluna para String
    df_part['TELEFONE'] = df_part['TELEFONE'].astype(object)
    
    #Criando novo DF chamado df
    df = pd.DataFrame(df_part)

    #Iterando sobre os indices contidos dentro do DataFrame, atribuindo os valores de cada indica a uma variável individual linha a linha
    for indice in range(len(df)):
        origem = df['ORIGEM'][indice]
        telefone = df['TELEFONE'][indice]
        data_bloqueio = df['DATA_BLOQUEIO'][indice]
        data_arquivo = df['DATA_ARQUIVO'][indice]
        empresa = df['EMPRESA'][indice]
        origem_pedido = df['ORIGEMPEDIDO'][indice]
        data_inicio_bloqueio = df['DATA_INICIO_BLOQUEIO'][indice]
    #Realizando inserção de dados na tabela cliente_nao_perturbe
        consulta = "INSERT IGNORE INTO cliente_nao_perturbe (origem, telefone, data_bloqueio, data_arquivo, empresa, OrigemPedido, data_inicio_bloqueio) VALUES (%s, %s, %s, %s, %s, %s, %s)"
        #Definindo valores que serão inseridos, de acordo com a posição marcada na consulta SQL
        valores = (origem, telefone, data_bloqueio, data_arquivo, empresa, origem_pedido, data_inicio_bloqueio)
        #Executando a função SQL, tendo como parametro a variável 'valores'
        cursor.execute(consulta, valores)
    
    #Realizando o commit, e confirmando as inserções realizadas no banco
    connection.commit()
    print(f"Dados no caminho {part_file} foram carregados com sucesso!\n")
#Fechando a conexão com o banco de dados.
connection.close()
print("Conexão Finalizada!")
